In [1]:
import os
import csv
import pandas as pd
import glob
os.add_dll_directory(r'c:\Program Files\IBM\IBM DATA SERVER DRIVER\bin')
import ibm_db
pd.options.mode.chained_assignment = None  

In [3]:
# 1) データベースから各パネルのデータを取得し、22KA662X60R_OutPutフォルダに保存します。

server   = "10.143.16.244"
database = "AYB_APPL"
username = "IOT_DATA"
password = "asd23fgh"
conn = ibm_db.connect("DATABASE=" + database + ";HOSTNAME=" + server + ";PORT=50000;PROTOCOL=TCPIP;UID=" + username + ";PWD=" + password + ";", "", "")

if conn:
    print("Connected")
    
list_db = ['B31A', 'B28', 'B20L', 'B15', 'B21', 'B36A', 'B29']  


query_template = """
        SELECT DISTINCT
            *
        FROM
            IOT_DATA.{}_ID_PROCESS tb
        WHERE
        tb.MATERIAL_ID IN (
                '22KA6620004502',
                '22KA6620004503',
                '22KA6620004504',
                '22KA6620004509',
                '22KA6620004510',
                '22KA6620004511',
                '22KA6620004512',
                '22KA6620004513',
                '22KA6620004514',
                '22KA6620004522'
            );
    """

for db in list_db:

    query = query_template.format(db)
    stmt = ibm_db.exec_immediate(conn, query)

    result_list = []
    row = ibm_db.fetch_assoc(stmt)
    while row:
        result_list.append(row)
        row = ibm_db.fetch_assoc(stmt)

    df = pd.DataFrame(result_list)
    df = df.dropna(axis=1, how='all')
    columns_to_drop = ['UNIT_NUM', 'PARTS_NUM', 'RECIPE_CODE', 'PN', 'JN', 'OPENO', 'LOTNO', 'LOT_NUM']
    df = df.drop(columns=[col for col in columns_to_drop if col in df.columns])

    df['MATERIAL_ID'] = df['MATERIAL_ID'].str.strip()

    cols = df.columns.tolist()  
  
    cols.insert(0, cols.pop(cols.index('MATERIAL_ID')))  
    cols.insert(1, cols.pop(cols.index('UNIT_NAME'))) 
    cols.insert(2, cols.pop(cols.index('MEASURED_DT')))  
    cols.insert(3, cols.pop(cols.index('ST_TIME'))) 
    cols.insert(4, cols.pop(cols.index('END_TIME')))
    cols.insert(5, cols.pop(cols.index('EQP_ID')))

    df = df[cols]  
 
    db_mapping = {
                'B31A': "B31A_デスミア",
                'B28': "B28_ABFラミ",
                'B20L': "B20L_PTNめっき",
                'B15': "B15_無電解銅めっき",
                'B21': "B21_レジスト剥離",
                'B36A': "B36A_回路形成",
                'B29': "B29_予備便化"
                }
    
    if db in db_mapping:
        mapped_db = db_mapping[db]

    root_path = r"c:\Users\00220401626\Desktop\VIASHIFT\22KA662X60R"
    full_path = os.path.join(root_path, mapped_db)
    df.to_excel(full_path + '.xlsx', index=False)

Connected


In [4]:
# 2) 各工程のデータを1行にまとめる　( 1行に1つのパネルの全工程データ)

folder = r'c:\Users\00220401626\Desktop\VIASHIFT\22KA662X60R\*.xlsx'  
excel_files = glob.glob(folder)
output_folder = r'c:\Users\00220401626\Desktop\VIASHIFT\Process' 

for file in excel_files:
    print(file)
    data = pd.read_excel(file)

    df = pd.DataFrame(target['MATERIAL_ID'].unique(), columns=['MATERIAL_ID'])

    if 'LOG_NUM' in data.columns and 'UNIT_NAME' in data.columns:
        data['UNIT_NAME'] = data['UNIT_NAME'].astype(str) + '_' + data['LOG_NUM'].astype(str)

    data.drop(columns=['LOG_NUM'], inplace=True)
    unique_units = data['UNIT_NAME'].unique()
   
    for unit in unique_units: 
        filtered_data = data[(data['UNIT_NAME'] == unit) & (data['MATERIAL_ID'].isin(target['MATERIAL_ID']))]
        df = pd.merge(df, filtered_data, on='MATERIAL_ID', how='left', suffixes=('', f'_{unit}'))

        df.dropna(axis=1, how='all', inplace=True)
        df = df.sort_values(by='ST_TIME')
        df = df.drop_duplicates('MATERIAL_ID', keep='first')
        df = df.sort_values(by='MATERIAL_ID')

    filename = os.path.basename(file).replace('.xlsx', '_processed.xlsx')
    output_path = os.path.join(output_folder, filename)

    df.to_excel(output_path, index=False)


c:\Users\00220401626\Desktop\VIASHIFT\22KA662X60R\B15_無電解銅めっき.xlsx
c:\Users\00220401626\Desktop\VIASHIFT\22KA662X60R\B20L_PTNめっき.xlsx
c:\Users\00220401626\Desktop\VIASHIFT\22KA662X60R\B21_レジスト剥離.xlsx
c:\Users\00220401626\Desktop\VIASHIFT\22KA662X60R\B28_ABFラミ.xlsx
c:\Users\00220401626\Desktop\VIASHIFT\22KA662X60R\B29_予備便化.xlsx
c:\Users\00220401626\Desktop\VIASHIFT\22KA662X60R\B31A_デスミア.xlsx
c:\Users\00220401626\Desktop\VIASHIFT\22KA662X60R\B36A_回路形成.xlsx


In [13]:
# 3) MATERIAL_ID, 平均値, 標準偏差, Cpk　データを作ります。

# ルートディレクトリ内のすべてのCSVファイルのパスを取得する関数
def get_csv_paths(root_dir):
    paths = []
    for subdir, dirs, files in os.walk(root_dir):
        for file in files:
            if file.endswith('.csv') and file.startswith('F'):
                paths.append(os.path.join(subdir, file))
    return paths


# CSVファイルを読み込む関数
def read_csv_file(file_path):
    separators = ['\t', ' ']
    for sep in separators:
        try:
            with open(file_path, 'r', encoding='utf-16') as file:
                reader = csv.reader(file, delimiter=sep)
                data = list(reader)
                if len(data[0]) > 1:
                    return data
        except UnicodeError:
            continue
    return None


# 読み込んだCSVデータを処理する関数
def process_csv_data(csv_data):
    df_list = []

    for row in csv_data:
        if len(row) == 4:
            df_list.append(pd.DataFrame({'Name1': [row[0]], 'Value1': [row[1]], 'Name2': [row[2]], 'Value2': [row[3]]}))
        elif len(row) == 2:
            df_list.append(pd.DataFrame({'Name1': [row[0]], 'Value1': [row[1]]}))

    if df_list:
        df = pd.concat(df_list, ignore_index=True)

        df1 = df[['Name1', 'Value1']].copy()
        df1.columns = ['Name', 'Value']

        df2 = df[['Name2', 'Value2']].dropna().copy()
        df2.columns = ['Name', 'Value']

        final = pd.concat([df1, df2])
        return final
    
    return None


# 処理したデータからCpkを計算する関数
def calculate_cpk(final, file_path):

    kyori = final[final['Name'].str.startswith('距離')]
    kyori['Value'] = kyori['Value'].astype(float)
    kyori['Value'] = kyori['Value'] *1000
    mean_value = kyori['Value'].mean()
    std_deviation = kyori['Value'].std()
    cpk = (13.5 - mean_value) / (3 * std_deviation)
  
    return pd.DataFrame({
        'MATERIAL_ID': ['22KA662000' + file_path[-8:-4]],
        '平均値': [mean_value],
        '標準偏差': [std_deviation],
        'Cpk': [cpk]
    })


# メイン処理
def main(root_dir):
    csv_paths = get_csv_paths(root_dir)
    df_list = []

    for path in csv_paths:
        csv_data = read_csv_file(path)
        if csv_data:
            final_df = process_csv_data(csv_data)
            final_df = calculate_cpk(final_df, path)
  
            if final_df is not None:
                df_list.append(final_df)

    target = pd.concat(df_list, ignore_index=True)
  
    return target


# Main
root_dir = r'C:\Users\00220401626\Desktop\VIASHIFT\E1G0.6600'
target = main(root_dir)
target = target.round(2)

target

,MATERIAL_ID,平均値,標準偏差,Cpk
0,22KA6620004502,4.51,2.38,1.26
1,22KA6620004503,2.89,2.32,1.52
2,22KA6620004504,3.60,1.99,1.66
3,22KA6620004509,4.18,2.08,1.49
4,22KA6620004510,2.78,1.99,1.80
5,22KA6620004511,4.65,2.41,1.22
6,22KA6620004512,3.05,1.99,1.75
7,22KA6620004513,4.14,1.69,1.84
8,22KA6620004514,3.06,1.41,2.47
9,22KA6620004522,4.43,2.36,1.28


In [12]:
# 4) Read_Time(各工程の間の時間)を取得する

asv = pd.read_excel(r'c:\Users\00220401626\Desktop\VIASHIFT\Read_Time\VIASHIFT_KI_20240628.xlsx', sheet_name= '22KA662X60R ※水色背景は設備データ有')

asv = asv[asv['入力日'] <= '2024-01-25']
asv = asv[asv['処理種'].isin(['COMP', 'OPST'])]
asv['入力時'] = pd.to_datetime(asv['入力時'], format='%H:%M:%S').dt.time
asv['Datetime'] = pd.to_datetime(asv['入力日'].dt.strftime('%Y-%m-%d') + ' ' + asv['入力時'].astype(str))
asv = asv[['Datetime', '処理種', 'オペNo.元', '工程名' ]]

opn = asv['オペNo.元'].unique()

results = []

for i in opn:
    max_time = asv.loc[(asv['オペNo.元'] == i) & (asv['処理種'] == 'OPST'), 'Datetime'].max()
    start_time_row = asv[(asv['オペNo.元'] == i) & (asv['処理種'] == 'OPST') & (asv['Datetime'] == max_time)]

    if not start_time_row.empty:
        comp_time_row = asv[(asv['Datetime'] < start_time_row['Datetime'].values[0]) & (asv['処理種'] == 'COMP')].max()
        
        if not comp_time_row.empty:
            time_difference = start_time_row['Datetime'].values[0] - comp_time_row['Datetime']
            results.append({
                'オペNo.元': i,
                'Start Time': start_time_row['Datetime'].values[0],
                'Comp Time': comp_time_row['Datetime'],
                'Time Difference': time_difference
            })

result = pd.DataFrame(results)
result = result[['オペNo.元', 'Time Difference']]

for index, row in result.iterrows():
    result[row['オペNo.元']] = row['Time Difference'] 
 
result.drop(columns=['オペNo.元', 'Time Difference'], inplace=True)
result = result.iloc[:10]
result['MATERIAL_ID'] = target['MATERIAL_ID']
read_time = result
read_time

,E1G0.5500,E1G0.4000,E1G0.3600,E1G0.3500,E1G0.3000,E1D0.6000,E1D0.5000,E1D0.4000,E1D0.3000,E1D0.2000,...,A0H0.3500,A0H0.3000,A0H0.1000,A0C1.7100,A0C1.6500,A0C1.6000,A0C1.5000,A0C1.3000,A0C1.1000,MATERIAL_ID
0,0 days 04:44:23,0 days 00:40:55,0 days 00:38:18,0 days 00:02:04,0 days 00:45:42,0 days 01:44:16,0 days 00:09:44,0 days 00:22:38,0 days 01:50:34,0 days 10:48:54,...,0 days 00:19:32,1 days 12:32:29,0 days 00:47:45,0 days 00:18:18,0 days 00:05:24,0 days 19:35:49,3 days 13:47:37,0 days 00:55:49,NaT,22KA6620004502
1,0 days 04:44:23,0 days 00:40:55,0 days 00:38:18,0 days 00:02:04,0 days 00:45:42,0 days 01:44:16,0 days 00:09:44,0 days 00:22:38,0 days 01:50:34,0 days 10:48:54,...,0 days 00:19:32,1 days 12:32:29,0 days 00:47:45,0 days 00:18:18,0 days 00:05:24,0 days 19:35:49,3 days 13:47:37,0 days 00:55:49,NaT,22KA6620004503
2,0 days 04:44:23,0 days 00:40:55,0 days 00:38:18,0 days 00:02:04,0 days 00:45:42,0 days 01:44:16,0 days 00:09:44,0 days 00:22:38,0 days 01:50:34,0 days 10:48:54,...,0 days 00:19:32,1 days 12:32:29,0 days 00:47:45,0 days 00:18:18,0 days 00:05:24,0 days 19:35:49,3 days 13:47:37,0 days 00:55:49,NaT,22KA6620004504
3,0 days 04:44:23,0 days 00:40:55,0 days 00:38:18,0 days 00:02:04,0 days 00:45:42,0 days 01:44:16,0 days 00:09:44,0 days 00:22:38,0 days 01:50:34,0 days 10:48:54,...,0 days 00:19:32,1 days 12:32:29,0 days 00:47:45,0 days 00:18:18,0 days 00:05:24,0 days 19:35:49,3 days 13:47:37,0 days 00:55:49,NaT,22KA6620004509
4,0 days 04:44:23,0 days 00:40:55,0 days 00:38:18,0 days 00:02:04,0 days 00:45:42,0 days 01:44:16,0 days 00:09:44,0 days 00:22:38,0 days 01:50:34,0 days 10:48:54,...,0 days 00:19:32,1 days 12:32:29,0 days 00:47:45,0 days 00:18:18,0 days 00:05:24,0 days 19:35:49,3 days 13:47:37,0 days 00:55:49,NaT,22KA6620004510
5,0 days 04:44:23,0 days 00:40:55,0 days 00:38:18,0 days 00:02:04,0 days 00:45:42,0 days 01:44:16,0 days 00:09:44,0 days 00:22:38,0 days 01:50:34,0 days 10:48:54,...,0 days 00:19:32,1 days 12:32:29,0 days 00:47:45,0 days 00:18:18,0 days 00:05:24,0 days 19:35:49,3 days 13:47:37,0 days 00:55:49,NaT,22KA6620004511
6,0 days 04:44:23,0 days 00:40:55,0 days 00:38:18,0 days 00:02:04,0 days 00:45:42,0 days 01:44:16,0 days 00:09:44,0 days 00:22:38,0 days 01:50:34,0 days 10:48:54,...,0 days 00:19:32,1 days 12:32:29,0 days 00:47:45,0 days 00:18:18,0 days 00:05:24,0 days 19:35:49,3 days 13:47:37,0 days 00:55:49,NaT,22KA6620004512
7,0 days 04:44:23,0 days 00:40:55,0 days 00:38:18,0 days 00:02:04,0 days 00:45:42,0 days 01:44:16,0 days 00:09:44,0 days 00:22:38,0 days 01:50:34,0 days 10:48:54,...,0 days 00:19:32,1 days 12:32:29,0 days 00:47:45,0 days 00:18:18,0 days 00:05:24,0 days 19:35:49,3 days 13:47:37,0 days 00:55:49,NaT,22KA6620004513
8,0 days 04:44:23,0 days 00:40:55,0 days 00:38:18,0 days 00:02:04,0 days 00:45:42,0 days 01:44:16,0 days 00:09:44,0 days 00:22:38,0 days 01:50:34,0 days 10:48:54,...,0 days 00:19:32,1 days 12:32:29,0 days 00:47:45,0 days 00:18:18,0 days 00:05:24,0 days 19:35:49,3 days 13:47:37,0 days 00:55:49,NaT,22KA6620004514
9,0 days 04:44:23,0 days 00:40:55,0 days 00:38:18,0 days 00:02:04,0 days 00:45:42,0 days 01:44:16,0 days 00:09:44,0 days 00:22:38,0 days 01:50:34,0 days 10:48:54,...,0 days 00:19:32,1 days 12:32:29,0 days 00:47:45,0 days 00:18:18,0 days 00:05:24,0 days 19:35:49,3 days 13:47:37,0 days 00:55:49,NaT,22KA6620004522


In [6]:
# 5) 製品情報を手動で作成する
extra_columns = {
    "製品サイズ": ["27.7x24.0"] * 10,
    "シートサイズ": ["271.35x234.25"] * 10,
    "製品取り数/pnl": [352] * 10,
    "部品数/Pcs": [0] * 10,
    "部品搭載": ["なし(ダミーパターン加工)"] * 10,
    "ABF材料": ["GX92R"] * 10,
    "予備硬化条件": ["レシピ16"] * 10,
    "デスミア条件": ["レシピ1"] * 10,
    "シードエッチ条件": ["レシピ3"] * 10,
}

product_info = pd.DataFrame(extra_columns)
product_info['MATERIAL_ID']= pd.DataFrame(target['MATERIAL_ID'].unique(), columns=['MATERIAL_ID'])
product_info

,製品サイズ,シートサイズ,製品取り数/pnl,部品数/Pcs,部品搭載,ABF材料,予備硬化条件,デスミア条件,シードエッチ条件,MATERIAL_ID
0,27.7x24.0,271.35x234.25,352,0,なし(ダミーパターン加工),GX92R,レシピ16,レシピ1,レシピ3,22KA6620004502
1,27.7x24.0,271.35x234.25,352,0,なし(ダミーパターン加工),GX92R,レシピ16,レシピ1,レシピ3,22KA6620004503
2,27.7x24.0,271.35x234.25,352,0,なし(ダミーパターン加工),GX92R,レシピ16,レシピ1,レシピ3,22KA6620004504
3,27.7x24.0,271.35x234.25,352,0,なし(ダミーパターン加工),GX92R,レシピ16,レシピ1,レシピ3,22KA6620004509
4,27.7x24.0,271.35x234.25,352,0,なし(ダミーパターン加工),GX92R,レシピ16,レシピ1,レシピ3,22KA6620004510
5,27.7x24.0,271.35x234.25,352,0,なし(ダミーパターン加工),GX92R,レシピ16,レシピ1,レシピ3,22KA6620004511
6,27.7x24.0,271.35x234.25,352,0,なし(ダミーパターン加工),GX92R,レシピ16,レシピ1,レシピ3,22KA6620004512
7,27.7x24.0,271.35x234.25,352,0,なし(ダミーパターン加工),GX92R,レシピ16,レシピ1,レシピ3,22KA6620004513
8,27.7x24.0,271.35x234.25,352,0,なし(ダミーパターン加工),GX92R,レシピ16,レシピ1,レシピ3,22KA6620004514
9,27.7x24.0,271.35x234.25,352,0,なし(ダミーパターン加工),GX92R,レシピ16,レシピ1,レシピ3,22KA6620004522


In [7]:
# 6) 1.製品情報/2.ReadTime/3.target(平均値、標準偏差、Cpk)をmergeする
product_info_read_time = pd.merge(product_info, read_time, on='MATERIAL_ID', how='left')
final = pd.merge(product_info_read_time, target, on='MATERIAL_ID', how='left')
final 

,製品サイズ,シートサイズ,製品取り数/pnl,部品数/Pcs,部品搭載,ABF材料,予備硬化条件,デスミア条件,シードエッチ条件,MATERIAL_ID,...,A0H0.1000,A0C1.7100,A0C1.6500,A0C1.6000,A0C1.5000,A0C1.3000,A0C1.1000,平均値,標準偏差,Cpk
0,27.7x24.0,271.35x234.25,352,0,なし(ダミーパターン加工),GX92R,レシピ16,レシピ1,レシピ3,22KA6620004502,...,0 days 00:47:45,0 days 00:18:18,0 days 00:05:24,0 days 19:35:49,3 days 13:47:37,0 days 00:55:49,NaT,4.51,2.38,1.26
1,27.7x24.0,271.35x234.25,352,0,なし(ダミーパターン加工),GX92R,レシピ16,レシピ1,レシピ3,22KA6620004503,...,0 days 00:47:45,0 days 00:18:18,0 days 00:05:24,0 days 19:35:49,3 days 13:47:37,0 days 00:55:49,NaT,2.89,2.32,1.52
2,27.7x24.0,271.35x234.25,352,0,なし(ダミーパターン加工),GX92R,レシピ16,レシピ1,レシピ3,22KA6620004504,...,0 days 00:47:45,0 days 00:18:18,0 days 00:05:24,0 days 19:35:49,3 days 13:47:37,0 days 00:55:49,NaT,3.60,1.99,1.66
3,27.7x24.0,271.35x234.25,352,0,なし(ダミーパターン加工),GX92R,レシピ16,レシピ1,レシピ3,22KA6620004509,...,0 days 00:47:45,0 days 00:18:18,0 days 00:05:24,0 days 19:35:49,3 days 13:47:37,0 days 00:55:49,NaT,4.18,2.08,1.49
4,27.7x24.0,271.35x234.25,352,0,なし(ダミーパターン加工),GX92R,レシピ16,レシピ1,レシピ3,22KA6620004510,...,0 days 00:47:45,0 days 00:18:18,0 days 00:05:24,0 days 19:35:49,3 days 13:47:37,0 days 00:55:49,NaT,2.78,1.99,1.80
5,27.7x24.0,271.35x234.25,352,0,なし(ダミーパターン加工),GX92R,レシピ16,レシピ1,レシピ3,22KA6620004511,...,0 days 00:47:45,0 days 00:18:18,0 days 00:05:24,0 days 19:35:49,3 days 13:47:37,0 days 00:55:49,NaT,4.65,2.41,1.22
6,27.7x24.0,271.35x234.25,352,0,なし(ダミーパターン加工),GX92R,レシピ16,レシピ1,レシピ3,22KA6620004512,...,0 days 00:47:45,0 days 00:18:18,0 days 00:05:24,0 days 19:35:49,3 days 13:47:37,0 days 00:55:49,NaT,3.05,1.99,1.75
7,27.7x24.0,271.35x234.25,352,0,なし(ダミーパターン加工),GX92R,レシピ16,レシピ1,レシピ3,22KA6620004513,...,0 days 00:47:45,0 days 00:18:18,0 days 00:05:24,0 days 19:35:49,3 days 13:47:37,0 days 00:55:49,NaT,4.14,1.69,1.84
8,27.7x24.0,271.35x234.25,352,0,なし(ダミーパターン加工),GX92R,レシピ16,レシピ1,レシピ3,22KA6620004514,...,0 days 00:47:45,0 days 00:18:18,0 days 00:05:24,0 days 19:35:49,3 days 13:47:37,0 days 00:55:49,NaT,3.06,1.41,2.47
9,27.7x24.0,271.35x234.25,352,0,なし(ダミーパターン加工),GX92R,レシピ16,レシピ1,レシピ3,22KA6620004522,...,0 days 00:47:45,0 days 00:18:18,0 days 00:05:24,0 days 19:35:49,3 days 13:47:37,0 days 00:55:49,NaT,4.43,2.36,1.28


In [8]:
# 7)  工程データをファイルごとに作成し、保存する

folder = r'c:\Users\00220401626\Desktop\VIASHIFT\Process\*.xlsx' 
output_folder = r'c:\Users\00220401626\Desktop\VIASHIFT\Result' 
excel_files = glob.glob(folder)

for file in excel_files:
    data = pd.read_excel(file)
    df = pd.merge(data, final, on='MATERIAL_ID', how='left')
    filename = os.path.basename(file)
    output_path = os.path.join(output_folder, filename)
    df.to_excel(output_path, index=False)
    print(output_path)

c:\Users\00220401626\Desktop\VIASHIFT\Result\B15_無電解銅めっき_processed.xlsx
c:\Users\00220401626\Desktop\VIASHIFT\Result\B20L_PTNめっき_processed.xlsx
c:\Users\00220401626\Desktop\VIASHIFT\Result\B21_レジスト剥離_processed.xlsx
c:\Users\00220401626\Desktop\VIASHIFT\Result\B28_ABFラミ_processed.xlsx
c:\Users\00220401626\Desktop\VIASHIFT\Result\B29_予備便化_processed.xlsx
c:\Users\00220401626\Desktop\VIASHIFT\Result\B31A_デスミア_processed.xlsx
c:\Users\00220401626\Desktop\VIASHIFT\Result\B36A_回路形成_processed.xlsx


In [9]:
# 8)  工程データを1つのファイルにまとめて保存する

folder = r'c:\Users\00220401626\Desktop\VIASHIFT\Process\*.xlsx' 
excel_files = glob.glob(folder)

df = pd.DataFrame(target['MATERIAL_ID'].unique(), columns=['MATERIAL_ID'])

for file in excel_files:
    data = pd.read_excel(file)
    df = pd.merge(df, data, on='MATERIAL_ID', how='left')

df = pd.merge(df, final, on='MATERIAL_ID', how='left')
df.to_excel(r'c:\Users\00220401626\Desktop\VIASHIFT\Result\All_Data.xlsx', index=False)

C:\Users\00220401626\AppData\Local\Temp\ipykernel_6536\2573767071.py:10: FutureWarning: Passing 'suffixes' which cause duplicate columns {'A80_x', 'A24_x', 'A42_x', 'A6_x', 'A56_x', 'A21_x', 'A61_x', 'A9_x', 'A34_x', 'A88_x', 'A82_x', 'A14_x', 'A79_x', 'A73_x', 'A43_x', 'EQP_ID_x', 'A47_x', 'A2_x', 'A32_x', 'A22_x', 'A37_x', 'A1_x', 'A35_x', 'A44_x', 'A33_x', 'A62_x', 'A65_x', 'A38_x', 'A49_x', 'A27_x', 'A36_x', 'A67_x', 'A8_x', 'A19_x', 'A85_x', 'A63_x', 'A40_x', 'A75_x', 'A60_x', 'A48_x', 'A13_x', 'A10_x', 'A16_x', 'A54_x', 'ST_TIME_x', 'A57_x', 'A23_x', 'A30_x', 'A25_x', 'A39_x', 'A18_x', 'A83_x', 'A46_x', 'A66_x', 'A71_x', 'A29_x', 'A64_x', 'A3_x', 'A28_x', 'A53_x', 'A86_x', 'A52_x', 'A15_x', 'A84_x', 'A69_x', 'A11_x', 'A58_x', 'END_TIME_x', 'A72_x', 'A68_x', 'A78_x', 'A45_x', 'A7_x', 'A74_x', 'A81_x', 'A31_x', 'A4_x', 'A51_x', 'A17_x', 'A59_x', 'A5_x', 'A55_x', 'A87_x', 'UNIT_NAME_x', 'MEASURED_DT_x', 'A77_x', 'A76_x', 'A70_x', 'A41_x', 'A50_x', 'A20_x', 'A26_x', 'A12_x'} in the r

In [ ]:
#フォルダ構成

# E1G0.6600    : 平均値、標準偏差、Cpk を計算するために元データ保存されるフォルダ
# 22KA662X60R  : データベースから取得したデータを保存するフォルダ
# Process      : 22KA662X60Rフォルダのデータを1パネルを1行にまとめて処理したファイルフォルダが保存されるフォルダ
# Result       : 全工程が1つのファイルにまとめられた、もしくは各工程がそれぞれ違うファイルにまとめられたデータが保存されるフォルダ
# Read_Time    : 各工程の間の時間ReadTimeを計算するために元データが保存されるフォルダ